### 1. Install dependencies and build the necessary cython scripts.
‘’‘
Note that we need to install some dependencies, including pytorch, psi4, netket, jax, scipy, sympy, numpy, h5py...
’‘’

In [ ]:
# !conda install -c psi4 psi4 -y
# !conda install -c pytorch pytorch -y
# !conda install -c netket netket -y
# !conda install -c jax jax -y
# !conda install -c scipy scipy -y
# !conda install -c sympy sympy -y
# !conda install -c numpy numpy -y
# !conda install -c h5py h5py -y
# !conda install -c matplotlib matplotlib -y

# !python src_cpp/setup.py build_ext --inplace --force


# find the below code in snqs_kpm.chem._psi4_template
# note that to run psi4 we need to import snqs_kpm...
# sys.path.append('absolute path of snqs_kpm')
# replace 'absolute path of snqs_kpm' with true snqs_kpm package path


### 2. Import packages

In [ ]:
from snqs_kpm import generate_mol_data
from snqs_kpm import KPM_AS, FCI_AS
import numpy as np
import subprocess
import matplotlib.pyplot as plt

### 3. Generate molecular data, qubit Hamiltonian and dipole moment operators

In [ ]:
mol_name = "H2O"
generate_mol_data(mol_name)

### 4. Set neural-network parameters

In [ ]:
n_layer = 1
n_hid = 64
n_layer_phase = 1
n_hid_phase = 128
n_train = 3000
s = 111

o = "./data/"+ mol_name
m = "./molecules/"+ mol_name

arc_command = [
    "-o", o,
    "-m", m,"-single_phase",
    "-n", "1",    
    "-output_freq", "25",
    "-save_freq", "-1",]

nn_command = [
    "-n_layer", f"{n_layer}",
    "-n_hid", f"{n_hid}",
    "-n_layer_phase", f"{n_layer_phase}",
    "-n_hid_phase", f"{n_hid_phase}",
    "-lr", "0.001",
    "-s", f"{s}",
    "-n_train", f"{n_train}"]

nqs_command = [
    "python", "-u", "-m", "snqs_kpm.nqs.run",
    # "-c",
    ] + arc_command + nn_command


### 5. Run NQS

In [ ]:
nqs = subprocess.run(nqs_command, check=True)

### 6. Run KPM

In [ ]:
kpm_command = [
    "python", "-u", "-m", "snqs_kpm.nqs.run_moments",
    "-c",
    "-NOM", "0",] + arc_command + nn_command

kpm = subprocess.run(kpm_command, check=True)

### 7. Calculate absorption spectrum

In [ ]:
e,i = KPM_AS(mol_name)

### 8. Calculate FCI absorption spectrum

In [ ]:
e_fci,i_fci = FCI_AS(mol_name)

### 9. Plot data

In [ ]:
plt.figure(dpi=350)
plt.plot(e,i,label='sNQS-KPM')
plt.plot(e_fci,i_fci/np.max(i_fci),'--',label='ED')
plt.xlim(0,50)
plt.legend()
plt.xlabel("Energy (eV)")
plt.ylabel("Intensity (a.u.)")